In [1]:
from gekko import GEKKO
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
import time

In [37]:
# Down sampling

sec = 140000
interval = 30

tcLab = pd.read_pickle('TCLab_MIMO_Data_002_10000.pkl') # Put the original name

downsampled_interval = interval # New interval (sec)



tcLab_resample = tcLab.iloc[::downsampled_interval, :]

# print(tcLab_resample) # Check the file

# tcLab_resample.to_pickle('Down_TCLab_MIMO_Data_001_1000_30s.pkl') # New name for down sample file

In [41]:
# File name
# Down_TCLab_MIMO_Data_001_1000_30s


data1 = tcLab_resample
# data1 = pd.DataFrame(np.array(tcLab).T, columns=['H1', 'H2', 'T1', 'T2']).iloc[70:]

In [58]:
data = data1[0:500]

In [83]:
data["T1"][0:4] = data["T1"][91]

/var/folders/cp/p82kw9zd4psdb4vl1wp3x09m0000gn/T/ipykernel_74199/3573714488.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [84]:
data["T2"][0:4] = data["T2"][91]

/var/folders/cp/p82kw9zd4psdb4vl1wp3x09m0000gn/T/ipykernel_74199/926328750.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [85]:
# file = open(file='/content/drive/MyDrive/LSTM_MPC/FOPDT/closed_loop_data_1200.pkl',mode='wb')
# dump([u1_store, u2_store, y1_store, y2_store, SP1, SP2], file)
# file.close()

fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(y=data["T1"],name="T1"), row=1, col=1)
fig.add_trace(go.Scatter(y=data["T2"],name="T2"), row=1, col=1)
fig.add_trace(go.Scatter(y=data["H1"],name="H1"), row=1, col=1)
fig.add_trace(go.Scatter(y=data["H2"],name="H2"), row=1, col=1)
# fig.add_trace(go.Scatter(y=u1_store,name="u1"), row=2, col=1)
# fig.add_trace(go.Scatter(y=u2_store,name="u2"), row=2, col=1)

# fig.update_layout(
#     template="plotly_white",
#     font_family="Times New Roman",
#     font_size = 20
#     )

# fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True, row=1 , col=1)
# fig.update_xaxes(title_text='Time',showline=True, linewidth=2, linecolor='black', mirror=True, row=2, col=1)
# fig.update_yaxes(title_text='y', showline=True, linewidth=2, linecolor='black', mirror=True, row=1, col=1)
# fig.update_yaxes(title_text='u', showline=True, linewidth=2, linecolor='black', mirror=True, row=2, col=1)
# fig.show()

# fig.write_image("fig1.eps")

In [86]:
m=GEKKO(remote=True)
tsim = len(data)
m.time = np.linspace(0,tsim-1,tsim)

# Process Gain
K11 = m.FV(1)
K12 = m.FV(.5) 
K21 = m.FV(.5) 
K22 = m.FV(1)

# Time Constant
tau11 = m.FV(5)
tau12 = m.FV(5) 
tau21 = m.FV(5) 
tau22 = m.FV(5) 

# Gekko variables for Input Output
T1 = m.CV(data["T1"].to_numpy())
T2 = m.CV(data["T2"].to_numpy())
H1 = m.MV(data["H1"].to_numpy())
H2 = m.MV(data["H2"].to_numpy())

x11 = m.SV(1)
x12 = m.SV(1)
x21 = m.SV(1)
x22 = m.SV(1)

K11.STATUS = 1
K12.STATUS = 1
K21.STATUS = 1
K22.STATUS = 1

tau11.STATUS = 1
tau12.STATUS = 1
tau21.STATUS = 1
tau22.STATUS = 1


#FOPDT Equation
m.Equation(x11.dt() + x11 / tau11 == K11 / tau11 * H1)
m.Equation(x12.dt() + x12 / tau12 == K12 / tau12 * H2)
m.Equation(T1 == x11 + x12)

m.Equation(x21.dt() + x21 / tau21 == K21 / tau21 * H1)
m.Equation(x22.dt() + x22 / tau22 == K22 / tau22 * H2)
m.Equation(T2 == x21 + x22)


m.options.IMODE = 5
m.solve(disp=True)

# u1_store = np.ones(tf)*u1.VALUE
# u2_store = np.ones(tf)*u2.VALUE
# y1_store = np.ones(tf)*y1.VALUE
# y2_store = np.ones(tf)*y2.VALUE


# plt.figure(0)
# plt.subplot(2,1,1)
# plt.plot(SP1, drawstyle='steps')
# plt.plot(SP2, drawstyle='steps')
# plt.plot(y1_store)
# plt.plot(y2_store)
# plt.legend(['SP1', 'SP2', 'y1', 'y2'])

# plt.subplot(2,1,2)
# plt.plot(u1_store, drawstyle='steps')
# plt.plot(u2_store, drawstyle='steps')
# plt.legend(['u1', 'u2'])
# plt.show()

apm 10.2.115.231_gk_model6 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           16
   Intermediates:            0
   Connections  :            0
   Equations    :            6
   Residuals    :            6
 
 Number of state variables:           8666
 Number of total equations: -         8658
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              8
 
 **********************************************
 Dynamic Estimation with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program contains Ip

  68  1.8017854e+05 5.64e-03 9.33e+00  -6.6 2.73e+01  -0.9 2.79e-03 6.12e-03f  1
  69  1.8005690e+05 5.50e-03 9.11e+00  -2.0 6.47e+00  -0.4 5.30e-02 2.49e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70  1.7993102e+05 5.49e-03 9.08e+00  -6.6 3.06e+01  -0.9 4.61e-03 5.63e-03f  1
  71  1.7985432e+05 5.63e-03 9.06e+00  -6.6 3.91e+01  -1.4 3.01e-03 3.10e-03f  1
  72  1.7970607e+05 5.76e-03 8.96e+00  -0.4 1.41e+01  -1.0 3.16e-02 1.46e-02f  1
  73  1.7958003e+05 6.00e-03 8.87e+00  -1.5 6.26e+01    -  4.41e-03 1.02e-02f  1
  74  1.7948653e+05 5.95e-03 8.77e+00  -6.6 4.98e+01    -  1.94e-02 1.17e-02f  1
  75  1.7936147e+05 5.91e-03 8.63e+00  -6.6 4.85e+01    -  2.31e-02 1.67e-02f  1
  76  1.7873233e+05 5.82e-03 8.50e+00  -6.7 5.17e+02    -  6.35e-04 1.54e-02f  1
  77  1.7850659e+05 5.79e-03 8.45e+00  -1.8 4.75e+02    -  1.79e-02 5.95e-03f  1
  78  1.7847574e+05 5.77e-03 1.25e+01  -6.7 3.92e+01    -  5.03e-02 3.74e-03f  1
  79  1.7837303e+05 6.92e-03

 162  1.7410883e+05 8.42e-03 1.46e+01  -0.9 1.42e+02  -1.8 1.74e-03 4.34e-03f  1
 163  1.7408142e+05 8.51e-03 1.45e+01  -0.9 2.64e+01  -1.4 7.07e-03 1.18e-02f  1
 164  1.7402142e+05 8.76e-03 1.41e+01  -0.9 2.07e+02  -1.9 2.10e-02 1.54e-02f  1
 165  1.7398972e+05 8.56e-03 1.38e+01  -0.9 4.44e+01  -1.5 5.97e-02 2.38e-02f  1
 166  1.7387008e+05 8.28e-03 1.33e+01  -0.9 4.16e+02  -1.9 3.87e-02 3.29e-02f  1
 167  1.7385647e+05 8.19e-03 1.32e+01  -6.8 5.08e+01  -1.5 7.34e-02 1.04e-02f  1
 168  1.7379082e+05 8.08e-03 1.30e+01  -0.6 1.06e+03  -2.0 1.54e-02 1.28e-02f  1
 169  1.7375481e+05 7.61e-03 1.23e+01  -0.2 2.21e+02  -1.6 7.78e-02 5.89e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170  1.7462398e+05 7.38e-03 2.44e+01   1.0 6.72e+01  -1.1 1.74e-03 3.04e-02f  1
 171  1.7448852e+05 7.07e-03 2.26e+01  -0.6 4.03e+02  -1.6 1.89e-03 4.67e-02f  1
 172  1.7434863e+05 6.75e-03 2.13e+01  -0.6 1.72e+02  -1.2 6.01e-02 5.74e-02f  1
 173  1.7429356e+05 7.51e-03

In [87]:
K11[-1]

0.078908668375

In [88]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(y=data["T1"],name="T1"), row=1, col=1)
fig.add_trace(go.Scatter(y=data["T2"],name="T2"), row=1, col=1)
fig.add_trace(go.Scatter(y=T1.value,name="T1_model"), row=1, col=1)
fig.add_trace(go.Scatter(y=T2.value,name="T2_model"), row=1, col=1)
fig.add_trace(go.Scatter(y=data["H1"],name="H1"), row=1, col=1)
fig.add_trace(go.Scatter(y=data["H2"],name="H2"), row=1, col=1)